In [1]:
#USING NEURAL NETWORK + BERT

import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm, datasets
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns

from sklearn import metrics

In [2]:
df = pd.read_excel(r"Data_600_Labeled_Final.xlsx")
df['innovation_num']=df.YN_INNOVATION.eq('yes').mul(1)

In [3]:
df = df.sort_values(by='YN_INNOVATION', ascending=False)

In [4]:
df.drop(df.tail(154).index, inplace = True)

In [5]:
df

,Unnamed: 0,an,title,snippet,body,full_text,"Does it mention an innovation (yes,no,unsure)?",What is the company?,"Where is company mentioned (title,body,snippet,all)?",What is the product name?,"Where is the product name mentioned(title,body,snippet,all)","What could indicate that this is a product (for use in our algorithms, e.g. TM,R,C,CAPITAL LETTERS, Quotes,Used with certain key phrases or words etc.)?","Features of the product, if mentioned",assignee,YN_INNOVATION,innovation_num
0,0,PNN0000020180426ee4p00006,Creaform and Metrologic Group Announce Worldwi...,Metrologic Group and Creaform have entered int...,Speed and volumetric accuracy\n\nThe CUBE-R™ e...,Creaform and Metrologic Group Announce Worldwi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,yes,1
515,515,PZON000020210113eh1d0003z,SwarmConnect Launches Video Conferencing for V...,SwarmConnect Launches Video Conferencing for V...,"SwarmConnect, in conjunction with DeepSkyConne...",SwarmConnect Launches Video Conferencing for V...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes,1
462,462,BWR0000020171003eda3000a5,Conga Launches Enterprise Edition of Conga Con...,Unveiled at Conga Connect user conference in L...,"Conga, an application provider for companies l...",Conga Launches Enterprise Edition of Conga Con...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aditi,yes,1
229,229,BWR0000020160426ec4q000dz,Skyjet Max Enables Instant Booking of Private ...,\n -- Enhanced Skyjet App features industry-...,This Smart News Release features multimedia. V...,Skyjet Max Enables Instant Booking of Private ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK,yes,1
227,227,INVWK00020200828eg8s0008l,unitQ; Drive Product Quality and Revenue from ...,2020 SEP 5 (VerticalNews) -- By a News Reporte...,unitQ helps teams find and fix product issues ...,unitQ; Drive Product Quality and Revenue from ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AK,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,38,MANAVG0020131007e9a7000ul,MongoDB gets USD150m in funding,7 October 2013 - US data management and big da...,The company said it will use the proceeds to s...,MongoDB gets USD150m in funding 7 October 2013...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no,0
34,34,JOENG00020150320eb3p0004g,Cognizant; Cognizant Drives Digital Transforma...,2015 MAR 25 (VerticalNews) -- By a News Report...,"Cognizant is bringing together its consulting,...",Cognizant; Cognizant Drives Digital Transforma...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no,0
33,33,INTEWK0020160920ec9k00026,Guidewire Software; Basler Versicherung Extend...,2016 SEP 20 (VerticalNews) -- By a News Report...,"""We were already happy with the earlier succes...",Guidewire Software; Basler Versicherung Extend...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no,0
29,29,ENTWK00020191129efbt0000c,Aristocrat Technologies; An Application for th...,2019 DEC 6 (VerticalNews) -- By a News Reporte...,The international trademark goods and services...,Aristocrat Technologies; An Application for th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diggy,no,0


In [6]:
## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
tokenized = df['full_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, 
                                                              truncation=True,max_length=512)))

In [8]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [9]:
np.array(padded).shape

(446, 512)

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(446, 512)

In [34]:
print(input_ids)

tensor([[  101, 13675,  5243,  ...,  2492, 10250,   102],
        [  101, 21708,  8663,  ...,  2489,  3979,   102],
        [  101, 26478,  2050,  ...,  1998,  8122,   102],
        ...,
        [  101,  5009, 20357,  ...,     0,     0,     0],
        [  101, 10488, 16033,  ...,     0,     0,     0],
        [  101, 10667,  9581,  ...,  5766,  1997,   102]])


In [11]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [12]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df['innovation_num']

In [17]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [31]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [32]:
nn_clf.fit(train_features, train_labels)

/home/am8hf/.conda/envs/AM_BERT/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='sgd')

In [33]:
nn_clf.score(test_features, test_labels)

0.7857142857142857